In [55]:
import geopandas as gpd
import pandas as pd
import ee
import geemap
ee.Authenticate()
ee.Initialize(project='pacaas-novas')

In [56]:
pacaas_area = gpd.read_file('C:/Users/ppgar/Downloads/QGIS/Pacaas_novos/pacaas_novas.geojson')

In [57]:
pacaas_area = pacaas_area.geometry.iloc[0].__geo_interface__

In [58]:
pacaas_area = ee.Geometry(pacaas_area)
buffer = pacaas_area.buffer(5000)
buffer_diff = buffer.difference(pacaas_area)

In [59]:
def calc_ndvi(image):
    nir = image.select('B8')
    red = image.select('B4')
    ndvi = nir.subtract(red).divide(nir.add(red)).rename('NDVI')
    return image.addBands(ndvi)

In [60]:
def mask_clouds(image):
    cloud_mask = image.select('QA60').eq(0)
    return image.updateMask(cloud_mask)

satelite_2016 = ee.ImageCollection('COPERNICUS/S2_HARMONIZED').filterDate('2016-05-01', '2016-09-30').map(mask_clouds).median().clip(buffer).select(['B11', 'B8', 'B4'])
satelite_2016_buffer = ee.ImageCollection('COPERNICUS/S2_HARMONIZED').filterDate('2016-05-01', '2016-09-30').map(mask_clouds).median().clip(buffer_diff).select(['B11', 'B8', 'B4'])

**Calculate the NDVI map for each year**

In [61]:
pacaas_hist = pd.DataFrame()
for year in range(2016,2025):
    ndvi_mean = calc_ndvi(ee.ImageCollection('COPERNICUS/S2_HARMONIZED').filterDate(f'{year}-06-01', f'{year}-09-30').map(mask_clouds).median().clip(pacaas_area)).select(['NDVI']).reduceRegion('mean')
    ndvi_buffer = calc_ndvi(ee.ImageCollection('COPERNICUS/S2_HARMONIZED').filterDate(f'{year}-06-01', f'{year}-09-30').map(mask_clouds).median().clip(buffer_diff)).select(['NDVI']).reduceRegion('mean')
    aux = pd.DataFrame({'Year': [year],
                        'Area_NDVI': [ndvi_mean.get('NDVI').getInfo()],
                        'Buffer_NDVI': [ndvi_buffer.get('NDVI').getInfo()],
                        'Area of Interest': [calc_ndvi(ee.ImageCollection('COPERNICUS/S2_HARMONIZED').filterDate(f'{year}-06-01', f'{year}-09-30').map(mask_clouds).median().clip(pacaas_area)).select('NDVI')],
                        'Buffer only': [calc_ndvi(ee.ImageCollection('COPERNICUS/S2_HARMONIZED').filterDate(f'{year}-06-01', f'{year}-09-30').map(mask_clouds).median().clip(buffer_diff)).select('NDVI')],
                        'Total Area': [calc_ndvi(ee.ImageCollection('COPERNICUS/S2_HARMONIZED').filterDate(f'{year}-06-01', f'{year}-09-30').map(mask_clouds).median().clip(buffer)).select('NDVI')]
                        })
    pacaas_hist = pd.concat([pacaas_hist, aux])

In [62]:
pacaas_hist.index = pacaas_hist['Year']
pacaas_hist.drop(columns = 'Year')

,Area_NDVI,Buffer_NDVI,Area of Interest,Buffer only,Total Area
Year,,,,,
2016,0.708851,0.709488,"ee.Image({\n ""functionInvocationValue"": {\n ...","ee.Image({\n ""functionInvocationValue"": {\n ...","ee.Image({\n ""functionInvocationValue"": {\n ..."
2017,0.700180,0.699647,"ee.Image({\n ""functionInvocationValue"": {\n ...","ee.Image({\n ""functionInvocationValue"": {\n ...","ee.Image({\n ""functionInvocationValue"": {\n ..."
2018,0.720925,0.721418,"ee.Image({\n ""functionInvocationValue"": {\n ...","ee.Image({\n ""functionInvocationValue"": {\n ...","ee.Image({\n ""functionInvocationValue"": {\n ..."
2019,0.635198,0.635114,"ee.Image({\n ""functionInvocationValue"": {\n ...","ee.Image({\n ""functionInvocationValue"": {\n ...","ee.Image({\n ""functionInvocationValue"": {\n ..."
2020,0.702197,0.703132,"ee.Image({\n ""functionInvocationValue"": {\n ...","ee.Image({\n ""functionInvocationValue"": {\n ...","ee.Image({\n ""functionInvocationValue"": {\n ..."
2021,0.655374,0.660780,"ee.Image({\n ""functionInvocationValue"": {\n ...","ee.Image({\n ""functionInvocationValue"": {\n ...","ee.Image({\n ""functionInvocationValue"": {\n ..."
2022,0.629308,0.629297,"ee.Image({\n ""functionInvocationValue"": {\n ...","ee.Image({\n ""functionInvocationValue"": {\n ...","ee.Image({\n ""functionInvocationValue"": {\n ..."
2023,0.681938,0.680261,"ee.Image({\n ""functionInvocationValue"": {\n ...","ee.Image({\n ""functionInvocationValue"": {\n ...","ee.Image({\n ""functionInvocationValue"": {\n ..."
2024,0.620735,0.629705,"ee.Image({\n ""functionInvocationValue"": {\n ...","ee.Image({\n ""functionInvocationValue"": {\n ...","ee.Image({\n ""functionInvocationValue"": {\n ..."


In [63]:
ndvi_2024 = pacaas_hist['Total Area'].loc[2024]
ndvi_2016 = pacaas_hist['Total Area'].loc[2016]

palette = ['red', 'orange', 'yellow', 'lightgreen', 'darkgreen']
mapa = geemap.Map()
mapa.set_center(-65, -11.3, 9)
mapa.addLayer(ndvi_2024.subtract(ndvi_2016), {'min': -0.5, 'max': 0.5, 'palette': palette}, 'NDVI Difference')
mapa

Map(center=[-11.3, -65], controls=(WidgetControl(options=['position', 'transparent_bg'], widget=SearchDataGUI(…

**Extract delta of change based on the NDVI difference between the 2016 and 2024**

In [64]:
def mask_region(ndvi_difference):
    significant_increase = ndvi_difference.gt(0.2)
    moderate_increase = ndvi_difference.gt(0.1).And(ndvi_difference.lte(0.2))
    stable = ndvi_difference.gt(-0.1).And(ndvi_difference.lt(0.1))
    moderate_decrease = ndvi_difference.gt(-0.2).And(ndvi_difference.lte(-0.1))
    significant_decrease = ndvi_difference.lte(-0.2)
    return significant_increase, moderate_increase, stable, moderate_decrease, significant_decrease

**Calculate the pixel area, to be converted to m² or hectares**

In [68]:
def calculate_area(area):
    pixel_area = ee.Image.pixelArea()
    return area.multiply(pixel_area)

In [73]:
pacaas_hist

,Year,Area_NDVI,Buffer_NDVI,Area of Interest,Buffer only,Total Area,Total Area in pixels
Year,,,,,,,
2016,2016,0.708851,0.709488,"ee.Image({\n ""functionInvocationValue"": {\n ...","ee.Image({\n ""functionInvocationValue"": {\n ...","ee.Image({\n ""functionInvocationValue"": {\n ...",NaN
2017,2017,0.700180,0.699647,"ee.Image({\n ""functionInvocationValue"": {\n ...","ee.Image({\n ""functionInvocationValue"": {\n ...","ee.Image({\n ""functionInvocationValue"": {\n ...",NaN
2018,2018,0.720925,0.721418,"ee.Image({\n ""functionInvocationValue"": {\n ...","ee.Image({\n ""functionInvocationValue"": {\n ...","ee.Image({\n ""functionInvocationValue"": {\n ...",NaN
2019,2019,0.635198,0.635114,"ee.Image({\n ""functionInvocationValue"": {\n ...","ee.Image({\n ""functionInvocationValue"": {\n ...","ee.Image({\n ""functionInvocationValue"": {\n ...",NaN
2020,2020,0.702197,0.703132,"ee.Image({\n ""functionInvocationValue"": {\n ...","ee.Image({\n ""functionInvocationValue"": {\n ...","ee.Image({\n ""functionInvocationValue"": {\n ...",NaN
2021,2021,0.655374,0.660780,"ee.Image({\n ""functionInvocationValue"": {\n ...","ee.Image({\n ""functionInvocationValue"": {\n ...","ee.Image({\n ""functionInvocationValue"": {\n ...",NaN
2022,2022,0.629308,0.629297,"ee.Image({\n ""functionInvocationValue"": {\n ...","ee.Image({\n ""functionInvocationValue"": {\n ...","ee.Image({\n ""functionInvocationValue"": {\n ...",NaN
2023,2023,0.681938,0.680261,"ee.Image({\n ""functionInvocationValue"": {\n ...","ee.Image({\n ""functionInvocationValue"": {\n ...","ee.Image({\n ""functionInvocationValue"": {\n ...",NaN
2024,2024,0.620735,0.629705,"ee.Image({\n ""functionInvocationValue"": {\n ...","ee.Image({\n ""functionInvocationValue"": {\n ...","ee.Image({\n ""functionInvocationValue"": {\n ...",NaN
